**Import necessary libraries**

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,mean_squared_error,classification_report,f1_score
from sklearn import preprocessing

In [0]:
filename=r'Datasets\Question-5\Train(1).csv'
df = pd.read_csv(filename)

**Data Exploration**

In [6]:
df.head()

Unnamed: 0                                               text author
0           0  The sight of the military restored hope to tho...    MWS
1           1  Just as the building was wiped out by a German...    HPL
2           2                    Do you wonder how it will seem?    HPL
3           3  My revenge is of no moment to you; yet, while ...    MWS
4           4  If he dares to reply, you can tell him from Lu...    EAP

In [7]:
df.describe()

Unnamed: 0
count  15663.000000
mean    7831.000000
std     4521.662968
min        0.000000
25%     3915.500000
50%     7831.000000
75%    11746.500000
max    15662.000000

Find the number of output labels and their relative count to check the **Class balance**

In [8]:
df.author.value_counts()

EAP    6316
MWS    4838
HPL    4509
Name: author, dtype: int64

In [9]:
plt.figure(figsize=(8,6))
df.groupby('author').text.count().plot.bar(ylim=0)
plt.show()


**Input**: Snippets from author's writing

**Output**: Identification of author given a text

**Method** : Supervised Classification using SVM

**Naive Implementation**

Designate **X** as features and **y** as labels

In [0]:
X=df.text
y=df.author

To run ML algorithm converting text to **BagofWords** representation.
Each word will represent a feature.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_CV = count_vect.fit_transform(X)
X_CV.shape

(15663, 23058)

In [12]:
X_CV[0]

<1x23058 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

Counting the number of words in each document will induce bias. It will give more weightage to longer documents than shorter documents. To avoid this, we can use frequency (TF - Term Frequencies).

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_TF = tfidf_transformer.fit_transform(X_CV)
X_TF.shape

(15663, 23058)

In [14]:
X_TF[0]

<1x23058 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

**Training and Testing Sets**

In [15]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)
y

array([2, 1, 1, ..., 1, 2, 2])

In [0]:
# from sklearn.preprocessing import OneHotEncoder

# onehotencoder1 = OneHotEncoder()
# y = onehotencoder1.fit_transform(y).toarray()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_TF,y,test_size=0.25, random_state=2489)

**Training Text Classification Model and Predicting Author**

In [0]:
from sklearn import svm 
SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto')
SVM.fit(X_train,y_train)

y_pred = SVM.predict(X_test)

**Evaluating the Model**

In [19]:
acc=accuracy_score(y_test, y_pred)*100
cm=confusion_matrix(y_test,y_pred)
cr=classification_report(y_test,y_pred)
print('Accuracy Score :\n',acc)
print('Confusion Matrix:\n',cm)
print('Classification Report :\n',cr)

Accuracy Score :
 80.84780388151175
Confusion Matrix:
 [[1306  132  133]
 [ 166  871   54]
 [ 188   77  989]]
Classification Report :
               precision    recall  f1-score   support

           0       0.79      0.83      0.81      1571
           1       0.81      0.80      0.80      1091
           2       0.84      0.79      0.81      1254

    accuracy                           0.81      3916
   macro avg       0.81      0.81      0.81      3916
weighted avg       0.81      0.81      0.81      3916



**Making the model persistent**

In [0]:
import pickle
with open('text_classifier_naive', 'wb') as picklefile:
    pickle.dump(SVM,picklefile)

**Load the model and check the test accuracy**

In [21]:
with open('text_classifier_naive', 'rb') as training_model:
    model = pickle.load(training_model)
    y_pred = model.predict(X_test)

    acc=accuracy_score(y_test, y_pred)*100
    cm=confusion_matrix(y_test,y_pred)
    cr=classification_report(y_test,y_pred)
    print('Accuracy Score :\n',acc)
    print('Confusion Matrix:\n',cm)
    print('Classification Report :\n',cr)

Accuracy Score :
 80.84780388151175
Confusion Matrix:
 [[1306  132  133]
 [ 166  871   54]
 [ 188   77  989]]
Classification Report :
               precision    recall  f1-score   support

           0       0.79      0.83      0.81      1571
           1       0.81      0.80      0.80      1091
           2       0.84      0.79      0.81      1254

    accuracy                           0.81      3916
   macro avg       0.81      0.81      0.81      3916
weighted avg       0.81      0.81      0.81      3916



**Improved prediction using data preprocessing**

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2',lowercase=True , ngram_range=(1, 2), stop_words='english')
X1 = tfidf.fit_transform(df.text).toarray()
y1 = df.author

In [23]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)
y

array([2, 1, 1, ..., 1, 2, 2])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X1,y1,test_size=0.25, random_state=2489)

In [0]:
from sklearn import svm 
SVM = svm.SVC(C=1.0, kernel='linear', gamma='auto')
SVM.fit(X_train,y_train)

In [0]:
with open('text_classifier_improved', 'wb') as picklefile:
    pickle.dump(SVM,picklefile)

In [0]:
y_pred = SVM.predict(X_train)

In [28]:
acc=accuracy_score(y_train, y_pred)*100
acc

92.61087937345705

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
# defining parameter range 
hyperparameters = {'C': [0.1, 1, 10, 100, 1000], 
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
			'kernel': ['rbf','linear','polynomial'],
            'degree': [2,3,5]} 

In [0]:
clf = RandomizedSearchCV(svm.SVC(), hyperparameters, random_state=1, cv=5, verbose=0, n_jobs=-1)

In [32]:
# Fit randomized search
best_model = clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
with open('text_classifier_RandomizedSearchCV', 'wb') as picklefile:
    pickle.dump(clf,picklefile)

In [41]:
# View best hyperparameters
print('Best Kernel:', best_model.best_estimator_.get_params()['kernel'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Kernel: rbf
Best C: 1000


In [34]:
# Predict target vector
best_model.predict(X_train)

array(['EAP', 'MWS', 'HPL', ..., 'MWS', 'HPL', 'MWS'], dtype=object)

In [0]:
y_pred=best_model.predict(X_train)

In [0]:
acc=accuracy_score(y_train, y_pred)*100
cm=confusion_matrix(y_train,y_pred)
cr=classification_report(y_train,y_pred)
print('Accuracy Score :\n',acc)
print('Confusion Matrix:\n',cm)
print('Classification Report :\n',cr)

In [0]:
# Predict target vector
y_pred=best_model.predict(X_test)

In [0]:
acc=accuracy_score(y_test, y_pred)*100
cm=confusion_matrix(y_test,y_pred)
cr=classification_report(y_test,y_pred)

print('Accuracy Score :\n',acc)
print('Confusion Matrix:\n',cm)
print('Classification Report :\n',cr)

In [43]:
clf.best_score_,clf.best_params_

(0.7719422841770605, {'C': 1000, 'degree': 3, 'gamma': 1, 'kernel': 'rbf'})